In [1]:
import time
from datetime import timedelta
import html

import pandas as pd
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

## Read csv files Java Answers

In [2]:
df = dd.read_csv('spotbugxmlreports_csv/spotbugReport*.csv', engine='python', error_bad_lines=False, warn_bad_lines=False, dtype=object)

#### Explore dataframe

In [3]:
## explore one patition
#one_pat_df = df.partitions[1].compute()

In [4]:
df

,instanceOccurrenceNum,instanceHash,rank,abbrev,category,priority,type,instanceOccurrenceMax,classname,end,endBytecode,isStatic,longMsg,msg,name,primary,signature,shortMsg,sourcepath,sourcefile,start,startBytecode,role
npartitions=577,,,,,,,,,,,,,,,,,,,,,,,
,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Start a Dask cluster using SLURM jobs as workers

In [5]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target" : False,  # avoid spilling to disk
        "distributed.worker.memory.spill" : False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-00:30",# walltime="0-00:50",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

Spawn between 20 to 100 workers and connect a client to be able use them.

In [6]:
#cluster.scale(n=20) # ask for 20 jobs or workers
# This also works with adaptive clusters. This automatically launches and kill workers based on load.
# we tell our cluster to autoscale between 10 and 20 workers depending on the load
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
#cluster.adapt(maximum_jobs=20)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.154:43906 Dashboard: http://192.168.94.154:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [7]:
df_init_len = df.shape[0].compute()
print('Initial number of volations {}'.format(df_init_len))

Initial number of volations 534978


In [8]:
df.head()

,instanceOccurrenceNum,instanceHash,rank,abbrev,category,priority,type,instanceOccurrenceMax,classname,end,...,msg,name,primary,signature,shortMsg,sourcepath,sourcefile,start,startBytecode,role
0,0,ce90bd5e82bd4d0d11cd96245d7fcadf,18,UrF,PERFORMANCE,2,URF_UNREAD_FIELD,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Unread field,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Code_10203353_10202514_4576_2,NaN,...,NaN,NaN,true,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Code_10203353_10202514_4576_2,2,...,NaN,NaN,NaN,NaN,NaN,Code_10203353_10202514_4576_2.java,Code_10203353_10202514_4576_2.java,1,NaN,NaN


In [9]:
#df.tail()

In [10]:
# get the headers
df = df[['category', 'type', 'sourcefile', 'msg', 'shortMsg']]

In [11]:
df.index.shape[0].compute()

534978

In [12]:
# Fill NaN cells with previous row values
#https://stackoverflow.com/questions/41212273/pandaspython-fill-empty-cells-with-with-previous-row-value
df = df.fillna(method='ffill')

In [13]:
# get specific needed columns 
df = df[['category', 'type', 'sourcefile', 'msg', 'shortMsg']]

In [14]:
# remove the records with null values 
df = df.dropna()

In [15]:
df.index.shape[0].compute()

534973

In [19]:
df.head()

,category,type,sourcefile,msg,shortMsg
5,PERFORMANCE,URF_UNREAD_FIELD,Code_10203353_10202514_4576_2.java,At Code_10203353_10202514_4576_2.java:[lines 1-2],Unread field
6,PERFORMANCE,URF_UNREAD_FIELD,Code_10203353_10202514_4576_2.java,In class Code_10203353_10202514_4576_2,Unread field
7,PERFORMANCE,URF_UNREAD_FIELD,Code_10203353_10202514_4576_2.java,In class Code_10203353_10202514_4576_2,Unread field
8,PERFORMANCE,URF_UNREAD_FIELD,Code_10203353_10202514_4576_2.java,In class Code_10203353_10202514_4576_2,Unread field
9,PERFORMANCE,URF_UNREAD_FIELD,Code_10203353_10202514_4576_2.java,In Code_10203353_10202514_4576_2.java,Unread field


In [20]:
df.tail()

,category,type,sourcefile,msg,shortMsg
800,PERFORMANCE,URF_UNREAD_FIELD,Code_997269_332079_2894_1.java,In class Code_997269_332079_2894_1,Unread field
801,PERFORMANCE,URF_UNREAD_FIELD,Code_997269_332079_2894_1.java,In Code_997269_332079_2894_1.java,Unread field
802,PERFORMANCE,URF_UNREAD_FIELD,Code_997269_332079_2894_1.java,Field Code_997269_332079_2894_1.hexArray,Unread field
803,PERFORMANCE,URF_UNREAD_FIELD,Code_997269_332079_2894_1.java,Field Code_997269_332079_2894_1.hexArray,Unread field
804,PERFORMANCE,URF_UNREAD_FIELD,Code_997269_332079_2894_1.java,At Code_997269_332079_2894_1.java:[line 2],Unread field


In [16]:
#df['lines'] = df['msg'].str.extract(r'line|s', expand=False)

In [17]:
nnn

NameError: name 'nnn' is not defined

### Test

In [ ]:
import numpy as np
import pandas as pd

import subprocess as sp

In [ ]:
csv_path = 'spotbugxmlreports_csv'

In [ ]:
folder = 'spotbugreports_wt_idx_csv'
mkdir_cmd = 'mkdir {}'.format(folder)
cmd = sp.run(
    mkdir_cmd, # command
    capture_output=True,
    text=True,
    shell=True
)

In [ ]:
# str.zfill(x) adds zeros in front of a number
array_id = '0'
file = 'spotbugReport{}'.format(array_id.zfill(3))
print(file)

In [ ]:
df = pd.read_csv('{}/{}.csv'.format(csv_path, file))

In [ ]:
df.columns

In [ ]:
df.index.shape[0]

In [ ]:
# Fill NaN cells with previous row values
#https://stackoverflow.com/questions/41212273/pandaspython-fill-empty-cells-with-with-previous-row-value
df.fillna(method='ffill', inplace=True)

In [ ]:
# get the headers
df = df[['category', 'type', 'sourcefile', 'msg', 'shortMsg']]

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.index.shape[0]

In [ ]:
#drop duplicate rows
df.drop_duplicates(inplace=True)

In [ ]:
df.index.shape[0]

In [ ]:
# use regex to extract lines where the errors occured in the msg
df[['bugline', 'bugline2']] = df['msg'].str.extract('((line|lines)\s+\d+\-?\d*)', expand=True)

In [ ]:
df.head()

In [ ]:
df = df[['category', 'type', 'sourcefile', 'shortMsg', 'bugline']]

In [ ]:
df = df.fillna('unident')

In [ ]:
df.head()

In [ ]:
# Rename
#df.rename(columns = {'index':'idx'}, inplace = True)
#df.to_csv('{}/{}.csv'.format(folder, file), index=False)